In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [128]:
urls = []
url_orig = 'https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=250&start='
for i in range(1,1000,250):
    urls.append(url_orig + str(i))
print(urls)

['https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=250&start=1', 'https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=250&start=251', 'https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=250&start=501', 'https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=250&start=751']


In [146]:
def get_titles(url):
    titles = []
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    show_images = soup.find_all('img', {"class":"loadlate"})
    for image in show_images:
        t = image['alt']
        titles.append(t)
    return titles

In [200]:
def get_genres(url):
    genres = []
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    genre_list = soup.find_all('span', {"class":"genre"})
    for g in genre_list:
        genre = g.text.strip().split(', ')
        genres.append(genre)
    return genres

In [222]:
def get_loglines(url):
    loglines = []
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    show_info = soup.find_all('div', {"class":"lister-item-content"})
    for show in show_info:
        logline = (show.find_all('p')[1].text.replace('\n','').strip())
        loglines.append(logline)
    return loglines

In [216]:
def get_years(url):
    years = []
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    year_list = soup.find_all('span', {"class":"lister-item-year text-muted unbold"})
    for y in year_list:
        year = y.text.strip().replace('(','').replace(')','')
        years.append(year)
    return years

In [213]:
def get_stars(url):
    all_stars = []
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    show_info = soup.find_all('div', {"class":"lister-item-content"})
    for show in show_info:
        stars = []
        star_list = show.find_all('p', {"class":""})[0].find_all('a')
        for s in star_list:
            stars.append(s.text)
        all_stars.append(stars)
    return all_stars

In [223]:
years = []
genre = []
stars = []
title = []
logline = []

for url in urls:
    title += get_titles(url)
    logline += get_loglines(url)
    genre += get_genres(url)
    years += get_years(url)
    stars += get_stars(url)

print(len(stars))



1000


In [224]:
shows_df = pd.DataFrame({'Title': title, 'years': years, 'genre': genre, 'stars': stars, 'logline': logline})
shows_df

In [229]:
logline_str = '\n'.join(logline)
f = open("loglines.txt", "w")
f.write(logline_str)
f.close()

In [230]:
shows_df.head

<bound method NDFrame.head of                 Title      years                           genre  \
0     Game of Thrones  2011–2019      [Action, Adventure, Drama]   
1        Breaking Bad  2008–2013        [Crime, Drama, Thriller]   
2     Stranger Things     2016–         [Drama, Fantasy, Horror]   
3             Friends  1994–2004               [Comedy, Romance]   
4    The Walking Dead  2010–2022       [Drama, Horror, Thriller]   
..                ...        ...                             ...   
995   Life Unexpected  2010–2011                         [Drama]   
996          TaleSpin  1990–1991  [Animation, Action, Adventure]   
997              Mars  2016–2018      [Adventure, Drama, Sci-Fi]   
998     Better Things  2016–2022                 [Comedy, Drama]   
999         Faking It  2014–2016        [Comedy, Drama, Romance]   

                                                 stars  \
0    [Emilia Clarke, Peter Dinklage, Kit Harington,...   
1    [Bryan Cranston, Aaron Paul, Ann